In [1]:
!pip install transformers
!pip install transformers[sentencepiece]

     |████████████████████████████████| 2.8 MB 5.3 MB/s 
     |████████████████████████████████| 895 kB 55.1 MB/s 
     |████████████████████████████████| 50 kB 6.0 MB/s 
     |████████████████████████████████| 3.3 MB 29.4 MB/s 
     |████████████████████████████████| 636 kB 56.0 MB/s 
  Attempting uninstall: pyyaml
    Found existing installation: PyYAML 3.13
    Uninstalling PyYAML-3.13:
      Successfully uninstalled PyYAML-3.13
     |████████████████████████████████| 1.1 MB 5.2 MB/s 


In [2]:
from transformers import pipeline
classifier = pipeline("zero-shot-classification",
                      model="vicgalle/xlm-roberta-large-xnli-anli")

Downloading:   0%|          | 0.00/734 [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/2.24G [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/25.0 [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/5.07M [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/150 [00:00<?, ?B/s]

In [4]:
sequence_to_classify = "building building 二番町センタービル "
candidate_labels = ['email','name','address','phone','serial_number','id','other','none','money']
identifiers = classifier(sequence_to_classify, candidate_labels)

identifiers

{'labels': ['address',
  'name',
  'id',
  'none',
  'serial_number',
  'money',
  'other',
  'phone',
  'email'],
 'scores': [0.6471672654151917,
  0.1725585162639618,
  0.10877256095409393,
  0.05074438080191612,
  0.0058462731540203094,
  0.00515188742429018,
  0.004379920195788145,
  0.0028095641173422337,
  0.0025696386583149433],
 'sequence': 'building building 二番町センタービル '}

In [5]:
from transformers import pipeline
import pandas as pd
def init_classifier():
    classifier = pipeline("zero-shot-classification",
                      model="vicgalle/xlm-roberta-large-xnli-anli")
    
def get_piis(text, top_n):
    tokenizer_classes = ['email','name','address','phone','serial_number','id/code','other','none','public information']
    identifiers = classifier(text, tokenizer_classes)
    topn_labels = identifiers['labels'][:top_n]
    topn_scores = identifiers['scores'][:top_n]
    return topn_labels,topn_scores

In [6]:
from google.colab import files
uploaded = files.upload()

Saving data_dictionary_sample.xls to data_dictionary_sample.xls


In [7]:
import io
df2 = pd.read_csv(io.BytesIO(uploaded['data_dictionary_sample.xls']))

In [8]:
df2

,field_name,definition,table_name,data_type,length,encryption_class,encrypted,primary_key,foreign_key,optional_mandatory,null_prec,derived_not_derived,logic,sample_value,source_field_name,source_data_filename
0,ez_account_id,Account ID of EZ_DB,m_account,varchar,30.0,ID,1.0,1,NaN,NaN,NaN,NaN,NaN,EZ-Acc-000000-0000000P,NaN,NaN
1,account_tp,Account Type,m_account,varchar,12.0,NaN,NaN,0,NaN,NaN,NaN,NaN,NaN,Person,NaN,NaN
2,prm_id,Partner ID(PRM),m_account,varchar,20.0,NaN,NaN,0,NaN,NaN,NaN,NaN,NaN,2-10XXB1X,NaN,NaN
3,ho_lbc,Customer Code in outside custmer database,m_account,varchar,20.0,ID,1.0,0,NaN,NaN,NaN,NaN,NaN,ez000000001,NaN,NaN
4,parent_lbc,Parent Customer Code in outside custmer database,m_account,varchar,20.0,ID,1.0,0,NaN,NaN,NaN,NaN,NaN,5686251,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
593,valid_from,record active from,t_case,datetime,NaN,NaN,NaN,0,NaN,NaN,NaN,NaN,NaN,13-08-2021 12.49,NaN,NaN
594,tier_active,lastest record,t_activity,smallint,NaN,NaN,NaN,0,NaN,NaN,NaN,NaN,NaN,1,NaN,NaN
595,tier_email_domain,email domain,t_activity,varchar,50.0,NaN,NaN,0,NaN,NaN,NaN,NaN,NaN,hp.com,NaN,NaN
596,valid_till,record active till,t_activity,datetime,NaN,NaN,NaN,0,NaN,NaN,NaN,NaN,NaN,13-08-2021 12.49,NaN,NaN


In [43]:
df2.table_name.unique()

array(['m_account', 'm_contact', 't_deal', 't_activity', 't_case',
       't_quote'], dtype=object)

In [72]:
for index, row in df2[df2['table_name']=='m_account'][10:].iterrows():
    #print(row['field_name'], row['definition'])
    fieldname = row['field_name']
    text = fieldname +' '+ str(row['definition']) + ' ' + str(row['sample_value'])
    print('text : ',text)
    piis = get_piis(text, 3)
    print('fieldname: ', fieldname, 'sample value: ', row['sample_value'] , 'classes: ', piis,'\n')

text :  hierarchy_tp Branch type(Head quarter or branch) JAPAN POST HOLDINGS - WW
fieldname:  hierarchy_tp sample value:  JAPAN POST HOLDINGS - WW classes:  (['none', 'name', 'id/code'], [0.39139726758003235, 0.18769489228725433, 0.08263561874628067]) 

text :  location_id Location id for Partner 70592611
fieldname:  location_id sample value:  70592611 classes:  (['id/code', 'none', 'public information'], [0.9345188140869141, 0.03388463705778122, 0.008935975842177868]) 

text :  zip_cd zip code 1028012
fieldname:  zip_cd sample value:  1028012 classes:  (['id/code', 'none', 'public information'], [0.8354923129081726, 0.07324791699647903, 0.04698823392391205]) 

text :  state state 東京都
fieldname:  state sample value:  東京都 classes:  (['address', 'none', 'name'], [0.7629992961883545, 0.1196429505944252, 0.06113843619823456]) 

text :  city city 千代田区
fieldname:  city sample value:  千代田区 classes:  (['address', 'name', 'none'], [0.9262790083885193, 0.02510158158838749, 0.018969571217894554])

KeyboardInterrupt: ignored

# Boolean QA Module

In [13]:
!pip install torch torchvision
!pip install pandas
!pip install numpy

!gsutil cp gs://boolq/train.jsonl .
!gsutil cp gs://boolq/dev.jsonl .
! pip install transformers datasets

Copying gs://boolq/train.jsonl...
/ [1 files][  6.2 MiB/  6.2 MiB]                                                
Operation completed over 1 objects/6.2 MiB.                                      
Copying gs://boolq/dev.jsonl...
/ [1 files][  2.1 MiB/  2.1 MiB]                                                
Operation completed over 1 objects/2.1 MiB.                                      
     |████████████████████████████████| 264 kB 5.0 MB/s 
     |████████████████████████████████| 243 kB 43.6 MB/s 
     |████████████████████████████████| 119 kB 58.8 MB/s 


In [9]:
import random
import torch
import numpy as np
import pandas as pd
from tqdm import tqdm
from torch.utils.data import TensorDataset, DataLoader, RandomSampler, SequentialSampler
from transformers import  AutoModelForSequenceClassification, AdamW

In [10]:
from transformers import AutoTokenizer
tokenizer = AutoTokenizer.from_pretrained("roberta-base")

Downloading:   0%|          | 0.00/481 [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/899k [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/456k [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/1.36M [00:00<?, ?B/s]

In [11]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [14]:
def encode_data(tokenizer, questions, passages, max_length):
    """Encode the question/passage pairs into features than can be fed to the model."""
    input_ids = []
    attention_masks = []

    for question, passage in zip(questions, passages):
        encoded_data = tokenizer.encode_plus(question, passage, max_length=max_length, pad_to_max_length=True, truncation_strategy="longest_first")
        encoded_pair = encoded_data["input_ids"]
        attention_mask = encoded_data["attention_mask"]

        input_ids.append(encoded_pair)
        attention_masks.append(attention_mask)

    return np.array(input_ids), np.array(attention_masks)

# Loading data
train_data_df = pd.read_json("/content/train.jsonl", lines=True, orient='records')
dev_data_df = pd.read_json("/content/dev.jsonl", lines=True, orient="records")

passages_train = train_data_df.passage.values
questions_train = train_data_df.question.values
answers_train = train_data_df.answer.values.astype(int)

passages_dev = dev_data_df.passage.values
questions_dev = dev_data_df.question.values
answers_dev = dev_data_df.answer.values.astype(int)

# Encoding data
max_seq_length = 256
input_ids_train, attention_masks_train = encode_data(tokenizer, questions_train, passages_train, max_seq_length)
input_ids_dev, attention_masks_dev = encode_data(tokenizer, questions_dev, passages_dev, max_seq_length)

train_features = (input_ids_train, attention_masks_train, answers_train)
dev_features = (input_ids_dev, attention_masks_dev, answers_dev)

Truncation was not explicitly activated but `max_length` is provided a specific value, please use `truncation=True` to explicitly truncate examples to max length. Defaulting to 'longest_first' truncation strategy. If you encode pairs of sequences (GLUE-style) with the tokenizer you can select this strategy more precisely by providing a specific strategy to `truncation`.
/usr/local/lib/python3.7/dist-packages/transformers/tokenization_utils_base.py:2204: FutureWarning: The `pad_to_max_length` argument is deprecated and will be removed in a future version, use `padding=True` or `padding='longest'` to pad to the longest sequence in the batch, or use `padding='max_length'` to pad to a max length. In this case, you can give a specific length with `max_length` (e.g. `max_length=45`) or leave max_length to None to pad to the maximal input size of the model (e.g. 512 for Bert).
  FutureWarning,


In [15]:
batch_size = 32

train_features_tensors = [torch.tensor(feature, dtype=torch.long) for feature in train_features]
dev_features_tensors = [torch.tensor(feature, dtype=torch.long) for feature in dev_features]

train_dataset = TensorDataset(*train_features_tensors)
dev_dataset = TensorDataset(*dev_features_tensors)

train_sampler = RandomSampler(train_dataset)
dev_sampler = SequentialSampler(dev_dataset)

train_dataloader = DataLoader(train_dataset, sampler=train_sampler, batch_size=batch_size)
dev_dataloader = DataLoader(dev_dataset, sampler=dev_sampler, batch_size=batch_size)

In [16]:
# Use a GPU if you have one available (Runtime -> Change runtime type -> GPU)
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

# Set seeds for reproducibility
random.seed(26)
np.random.seed(26)
torch.manual_seed(26)



model = AutoModelForSequenceClassification.from_pretrained("roberta-base")
model.to(device) # Send the model to the GPU if we have one

learning_rate = 1e-5
optimizer = AdamW(model.parameters(), lr=learning_rate, eps=1e-8)

Downloading:   0%|          | 0.00/501M [00:00<?, ?B/s]

Some weights of the model checkpoint at roberta-base were not used when initializing RobertaForSequenceClassification: ['roberta.pooler.dense.weight', 'roberta.pooler.dense.bias', 'lm_head.dense.weight', 'lm_head.layer_norm.weight', 'lm_head.decoder.weight', 'lm_head.bias', 'lm_head.dense.bias', 'lm_head.layer_norm.bias']
- This IS expected if you are initializing RobertaForSequenceClassification from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing RobertaForSequenceClassification from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
Some weights of RobertaForSequenceClassification were not initialized from the model checkpoint at roberta-base and are newly initialized: ['classifier.out_proj.weight', 'classi

In [ ]:
epochs = 2
grad_acc_steps = 1
train_loss_values = []
dev_acc_values = []

for _ in tqdm(range(epochs), desc="Epoch"):

  # Training
  epoch_train_loss = 0 # Cumulative loss
  model.train()
  model.zero_grad()

  for step, batch in enumerate(train_dataloader):

      input_ids = batch[0].to(device)
      attention_masks = batch[1].to(device)
      labels = batch[2].to(device)     

      outputs = model(input_ids, token_type_ids=None, attention_mask=attention_masks, labels=labels)

      loss = outputs[0]
      loss = loss / grad_acc_steps
      epoch_train_loss += loss.item()

      loss.backward()
      
      if (step+1) % grad_acc_steps == 0: # Gradient accumulation is over
        torch.nn.utils.clip_grad_norm_(model.parameters(), 1.0) # Clipping gradients
        optimizer.step()
        model.zero_grad()

  epoch_train_loss = epoch_train_loss / len(train_dataloader)          
  train_loss_values.append(epoch_train_loss)
  
  # Evaluation
  epoch_dev_accuracy = 0 # Cumulative accuracy
  model.eval()

  for batch in dev_dataloader:
    
    input_ids = batch[0].to(device)
    attention_masks = batch[1].to(device)
    labels = batch[2]
                
    with torch.no_grad():        
        outputs = model(input_ids, token_type_ids=None, attention_mask=attention_masks)
                    
    logits = outputs[0]
    logits = logits.detach().cpu().numpy()
    
    predictions = np.argmax(logits, axis=1).flatten()
    labels = labels.numpy().flatten()
    
    epoch_dev_accuracy += np.sum(predictions == labels) / len(labels)

  epoch_dev_accuracy = epoch_dev_accuracy / len(dev_dataloader)
  dev_acc_values.append(epoch_dev_accuracy)

Epoch:   0%|          | 0/2 [00:00<?, ?it/s]

In [ ]:
import seaborn as sns
import matplotlib.pyplot as plt

sns.set()

plt.plot(train_loss_values, label="train_loss")

plt.xlabel("Epoch")
plt.ylabel("Loss")
plt.title("Training Loss")
plt.legend()
plt.xticks(np.arange(0, 5))
plt.show()

In [ ]:
plt.plot(dev_acc_values, label="dev_acc")

plt.xlabel("Epoch")
plt.ylabel("Accuracy")
plt.title("Evaluation Accuracy")
plt.legend()
plt.xticks(np.arange(0, 5))
plt.show()

In [15]:
PATH = '/content/drive/MyDrive/finetuned_models/finetune_roberta'
torch.save(model, PATH)

In [16]:
model=torch.load(PATH)

In [21]:
def predict(question, passage):
  sequence = tokenizer.encode_plus(question, passage, return_tensors="pt")['input_ids'].to(device)
  
  logits = model(sequence)[0]
  probabilities = torch.softmax(logits, dim=1).detach().cpu().tolist()[0]
  proba_yes = round(probabilities[1], 2)
  proba_no = round(probabilities[0], 2)

  print(f"Question: {question}, Yes: {proba_yes}, No: {proba_no}")
  
text = """
location_id Location id for Partner 70592611
"""
 

questions = [
"Is it a Personally identifiable information field?"
]

 
for s_question in questions:
  predict(s_question, text)

 

Question: Is it a Personally identifiable information field?, Yes: 0.42, No: 0.58


In [26]:
questions = [
"Is it a Personally identifiable information field?"
]

for index, row in df2[df2['table_name']=='contact'].iterrows():
    #print(row['field_name'], row['definition'])
    fieldname = row['field_name']
    text = fieldname +' '+ str(row['definition']) + ' ' + str(row['sample_value'])
    print('text : ',text)
    piis = get_piis(text, 3)
    print('fieldname: ', fieldname, 'sample value: ', row['sample_value'] , 'classes: ', piis)
    predict(s_question, text,'\n')
